<img src="images/JHI_STRAP_Web.png" style="width: 150px; float: right;">
# 01 - Introduction to sequence data and bioinformatics

## Table of Contents

1. [Genomic context](#genomic_context)
2. [Genomic context on the NCBI website](genomic_context_ncbi)
3. [Genomic context using Artemis](genomic_context_art)

<a id="genomic_context"></a>
## Genomic context

You can look directly at the GenBank file see what genes or other features are annotated nearby, and deduce if these genes are part of a gene locus etc. However, this is much easier to see graphically.

<a id="genomic_context_ncbi"></a>
## Genomic context on the NCBI website

You might have noticed on the NCBI website display that for the gene ``ECA0662`` we searched for there was a clickable database cross-reference link, [GeneID:2881615](https://www.ncbi.nlm.nih.gov/sites/entrez?db=gene&cmd=Retrieve&dopt=full_report&list_uids=2881615) - that will show you the gene on the ``NC_004547.2`` genome visually:

![Genomic context for GeneID:2881615](./images/genome_context_GeneID_2881615.png)

You can try zooming out etc to examine what else is nearby - is your gene of interest part of an operon? Are the repeat regions nearby or anything else which might be important like an integrated phage?

However, nice as that is, it is often useful to be able to vizualise the genomic context out with the NCBI website. You might be working on a draft genome which has not been published yet?

For this we will open the downloaded GenBank plain text files in the [Sanger Institute's tool Artemis](http://www.sanger.ac.uk/science/tools/artemis), which is able to view and edit annotation files.

<a id="genomic_context_art"></a>
## Genomic context using Artemis

You should have previously download this GenBank file, in which case you can load it with Artemis via the command ``art`` as follows:

```bash
$ cd ~/Teaching-IBioIC-Intro-to-Bioinformatics/01-introduction/data
$ ls *.gbk
NC_004547.gbk
$ art NC_004547.gbk &
```

Artemis is a graphical tool written in Java, and should pop up a separate window from this Notebook - or from your terminal window if you run it directly at the command line. After a couple of harmless warnings, you should get a window like this:

![Artemis with NC_004547.gbk loaded](images/artemis_NC_004547_default.png)

The main window should have three regions,
- Zoomed out view (showing genes as coloured bars)
- Zoomed in view (showing individual bases and translated reading frames)
- List of features

Using the menu entry "Goto", "Navigator..." (or by pressing Ctrl+g), which pops up an "Artemis Navigator" window, tell Artemis to goto feature with the qualifier value ``ECA0662`` (the old locus ID), and you should see something like this:

![Artemis with NC_004547.gbk loaded showing ECA0662 gene](images/artemis_NC_004547_goto.png)

Notice a while gene feature has been outlined in black. Click "Goto" again, and the plane blue CDS feature should be highlighted:

![Artemis with NC_004547.gbk loaded showing ECA0662 CDS](images/artemis_NC_004547_goto2.png)

Clicking "Goto" a third time should say "text not found".

This gene is on the reverse complement strand - Artemis shows these features on the bottom half of the track, with the arrow pointing left. Looking to the left (downstream) there is a gene on the opposite forward strand. Looking to the right (upstream) there is a small gene on the same strand, then a small steel blue repeat region.

In the bottom pane Artemis shows any note in the annotation for each feature - this repeat region is described as a genomic island repeat, 3-prime fragment of tRNA Met.

If you double click on any feature, Artemis will jump to center it.

With a feature selected, pressing ``E`` brings up a window to edit the annotation of that feature. This is the quickest way to see things like the product or other information.

Here's the edit view for the ``ECA0662`` CDS feature, which should be very familiar from looking at the GenBank file format entry:

![Artemis with NC_004547.gbk loaded showing ECA0662 CDS](images/artemis_NC_004547_edit.png)


### Resources

* [Sanger Institute's tool Artemis](http://www.sanger.ac.uk/science/tools/artemis)
 